<a href="https://colab.research.google.com/github/MsAsterisk1/ERM_GDRO/blob/main/civil_comments_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!nvidia-smi -L  

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-352a2569-526e-4102-9949-ec39e9fbec74)


In [3]:
!git clone https://github.com/MsAsterisk1/ERM_GDRO.git
%cd /content/ERM_GDRO

fatal: destination path 'ERM_GDRO' already exists and is not an empty directory.
/content/ERM_GDRO


In [4]:
# !git pull

In [5]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import pandas as pd
from utils.process_data_utils import  get_CivilComments_DataLoaders, get_CivilComments_Datasets, get_CivilComments_df
from loss import ERMLoss, GDROLoss
from train_eval import train, evaluate, train_epochs
from transformers import DistilBertTokenizer, DistilBertModel
import torch
from torch import nn

Now lets try getting Bert working

In [7]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cuda'

In [8]:
df = get_CivilComments_df()

In [9]:
datasets = get_CivilComments_Datasets(CC_df=df, device=DEVICE)

In [10]:
train,cv,test=  get_CivilComments_DataLoaders(datasets=datasets)

This is how we would initialize model

In [11]:
class BertClassifier(nn.Module):

    def __init__(self, classes=2, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, classes)
        self.relu = nn.ReLU()

    def forward(self, X):
        input_id, mask = X
        pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)[0][:,0]
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [12]:
model = BertClassifier().to(DEVICE)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.00005, weight_decay=0.01)
epochs = 5
loss_fn = ERMLoss(model, torch.nn.CrossEntropyLoss())
# loss_fn = GDROLoss(model, torch.nn.CrossEntropyLoss(), 0.01, 8, vector_subclass=True)

train_epochs(epochs, train, test, model, loss_fn, optimizer, vector_subclass=True, verbose=True, num_subclasses=8)


Epoch 1 / 5
Epoch 2 / 5


In [ ]:
torch.save(model.state_dict(), './weights_ERM')


In [ ]:
from google.colab import files
files.download('./weights_ERM')